In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
from time import time
import datetime
from scipy.optimize import minimize, fsolve

from collections import Counter

from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import QuantileTransformer, RobustScaler

import torch
import torch.nn as nn
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
from transformers import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup

import copy

import seaborn as sns
import plotly.express as px

import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


In [2]:
RANDOM_SEED=42

def seed_everything(seed=RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=RANDOM_SEED)

# Loading the data

In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
test_data = pd.read_csv('../input/lish-moa/test_features.csv')
train_drug_ids = pd.read_csv('../input/lish-moa/train_drug.csv') 

train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

train_data = train_features.merge(train_targets, on='sig_id', how='left')
train_data = train_data.merge(train_drug_ids, on='sig_id', how='left') 

In [4]:
target_columns = [c for c in train_targets.columns if c != 'sig_id']
gene_features = [col for col in train_features.columns if col.startswith('g-')]
cell_features = [col for col in train_features.columns if col.startswith('c-')]
feature_columns = gene_features + cell_features

# Cross validation strategy

In [5]:
def create_cross_validation_strategy(data, targets, FOLDS, SEED):

    vc = data.drug_id.value_counts()
    
#     vc1 = vc.loc[(vc==6)|(vc==12)|(vc==18)].index.sort_values()
#     vc2 = vc.loc[(vc!=6)&(vc!=12)&(vc!=18)].index.sort_values()
    
    vc1 = vc.loc[vc <= 19].index.sort_values()
    vc2 = vc.loc[vc > 19].index.sort_values()

    dct1 = {} 
    dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

    tmp = data.groupby('drug_id')[targets].mean().loc[vc1]
    
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    tmp = data.loc[data.drug_id.isin(vc2)].reset_index(drop=True)
    
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)

    # ASSIGN FOLDS
    data['fold'] = data.drug_id.map(dct1)
    data.loc[data.fold.isna(),'fold'] = data.loc[data.fold.isna(),'sig_id'].map(dct2)
    data.fold = data.fold.astype('int8')
    
    return data

# Modeling

In [6]:
class MoaDataset:
    def __init__(self, dataset_df, gene_features, cell_features, target_ids):
        self.dataset_df = dataset_df
        self.target_ids = target_ids
    
        self.gene_features = self.dataset_df[gene_features].values
        self.cell_features = self.dataset_df[cell_features].values
        self.targets = None
    
        if self.target_ids is not None:
            self.targets = self.dataset_df[target_ids].values

    def __len__(self):
        return len(self.dataset_df)
    
    def number_of_features(self):
        return self.gene_features.shape[1], self.cell_features.shape[1]

    def __getitem__(self, item):
        dataset_sample = {}

        dataset_sample['genes'] = torch.tensor(self.gene_features[item, :], dtype=torch.float)
        dataset_sample['cells'] = torch.tensor(self.cell_features[item, :], dtype=torch.float)
        dataset_sample['sig_id'] = self.dataset_df.loc[item, 'sig_id']

        if self.target_ids is not None:
            dataset_sample['y'] = torch.tensor(self.targets[item, :], dtype=torch.float)

        return dataset_sample

In [7]:
class ModelConfig:
    def __init__(self, number_of_genes, number_of_cells, number_of_targets):
        self.number_of_genes = number_of_genes
        self.number_of_cells = number_of_cells
        self.number_of_targets = number_of_targets

In [8]:
class MoaModelBlock(nn.Module):
    def __init__(self, num_in, num_out, dropout, weight_norm=False, ):
        super().__init__()
        self.batch_norm = nn.BatchNorm1d(num_in)
        self.dropout = nn.Dropout(dropout)
        
        if weight_norm:
            self.linear = nn.utils.weight_norm(nn.Linear(num_in, num_out))
        else:
            self.linear = nn.Linear(num_in, num_out)
        
        self.activation = nn.PReLU(num_out)
        
        
    def forward(self, x):
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.activation(x)
        return x

class MoaEncodeBlock(nn.Module):
    def __init__(self, num_in, num_out, dropout, weight_norm=False):
        super().__init__()
        self.batch_norm = nn.BatchNorm1d(num_in)
        self.dropout = nn.Dropout(dropout)
        
        if weight_norm:
            self.linear = nn.utils.weight_norm(nn.Linear(num_in, num_out))
        else:
            self.linear = nn.Linear(num_in, num_out)

    def forward(self, x):
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = self.linear(x)
        return x

In [9]:
class MoaModel_V1(nn.Module):
    def __init__(self, model_config):
        super().__init__()
        total_features = model_config.number_of_genes + model_config.number_of_cells
        dropout = 0.15
        hidden_size = 1024
        
        self.block1 = MoaModelBlock(total_features, 2048, dropout)
        self.block2 = MoaModelBlock(2048, 1024, dropout)
        self.model = nn.Sequential(
                          nn.BatchNorm1d(1024),
                          nn.Dropout(dropout),
                          nn.Linear(1024, model_config.number_of_targets))

    def forward(self, data):
        x_genes = data['genes']
        x_cells = data['cells']
        
        x = torch.cat((x_genes, x_cells), dim=1)
        x = x.to(DEVICE)
        
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.model(x)
        return x

In [10]:
class MoaModel_V2(nn.Module):
    def __init__(self, model_config):
        super().__init__()
        dropout = 0.15
        hidden_size = 512
        
        self.genes_encoder = MoaEncodeBlock(model_config.number_of_genes, 128, dropout)
            
        self.cells_encoder = MoaEncodeBlock(model_config.number_of_cells, 32, dropout)
        
        out_encodings = 128 + 32
    
        self.block1 = MoaModelBlock(128, hidden_size, dropout)
        self.block2 = MoaModelBlock(32, hidden_size, dropout)
        
        self.block3 = MoaModelBlock(hidden_size, hidden_size, dropout)
        self.block4 = MoaModelBlock(hidden_size, hidden_size, dropout)

        self.model = nn.Sequential(
                          nn.BatchNorm1d(hidden_size),
                          nn.Dropout(dropout),
                          nn.Linear(hidden_size, model_config.number_of_targets))

    def forward(self, data):
        x_genes = data['genes'].to(DEVICE)
        x_cells = data['cells'].to(DEVICE)
        
        encoded_genes = self.genes_encoder(x_genes)
        encoded_cells = self.cells_encoder(x_cells)
        
        x_genes = self.block1(encoded_genes)
        x_cells = self.block2(encoded_cells)

        x = self.block3(x_genes + x_cells)
        x = self.block4(x)
        x = self.model(x)
        
        return x

In [11]:
class MoaModel_V3(nn.Module):
    def __init__(self, model_config):
        super().__init__()
        dropout = 0.15
        hidden_size = 512
        
        self.genes_encoder = MoaEncodeBlock(model_config.number_of_genes, 128, dropout)
            
        self.cells_encoder = MoaEncodeBlock(model_config.number_of_cells, 32, dropout)
            
        out_encodings = 128 + 32
    
        self.block1 = MoaModelBlock(out_encodings, hidden_size, dropout)
        self.block2 = MoaModelBlock(hidden_size, hidden_size, dropout)
        self.model = nn.Sequential(
                          nn.BatchNorm1d(hidden_size),
                          nn.Dropout(dropout),
                          nn.Linear(hidden_size, model_config.number_of_targets))

    def forward(self, data):
        x_genes = data['genes'].to(DEVICE)
        x_cells = data['cells'].to(DEVICE)
        
        encoded_genes = self.genes_encoder(x_genes)
        encoded_cells = self.cells_encoder(x_cells)
               
        x = torch.cat((encoded_genes, encoded_cells), 1)
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.model(x)
        return x

In [12]:
# class MoaModel_V4(nn.Module):
#         def __init__(self, model_config):
#             super(MoaModel_V4, self).__init__()
            
#             num_features = model_config.number_of_genes + model_config.number_of_cells
#             hidden_size = 4096
            
#             cha_1 = 256
#             cha_2 = 512
#             cha_3 = 512

#             cha_1_reshape = int(hidden_size/cha_1)
#             cha_po_1 = int(hidden_size/cha_1/2)
#             cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

#             self.cha_1 = cha_1
#             self.cha_2 = cha_2
#             self.cha_3 = cha_3
#             self.cha_1_reshape = cha_1_reshape
#             self.cha_po_1 = cha_po_1
#             self.cha_po_2 = cha_po_2

#             self.batch_norm1 = nn.BatchNorm1d(num_features)
#             self.dropout1 = nn.Dropout(0.1)
#             self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

#             self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
#             self.dropout_c1 = nn.Dropout(0.1)
#             self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

#             self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

#             self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
#             self.dropout_c2 = nn.Dropout(0.1)
#             self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

#             self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
#             self.dropout_c2_1 = nn.Dropout(0.3)
#             self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

#             self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
#             self.dropout_c2_2 = nn.Dropout(0.2)
#             self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

#             self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

#             self.flt = nn.Flatten()

#             self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
#             self.dropout3 = nn.Dropout(0.2)
#             self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, model_config.number_of_targets))

#         def forward(self, data):
            
#             x_genes = data['genes']
#             x_cells = data['cells']

#             x = torch.cat((x_genes, x_cells), dim=1)
#             x = x.to(DEVICE)

#             x = self.batch_norm1(x)
#             x = self.dropout1(x)
#             x = F.celu(self.dense1(x), alpha=0.06)

#             x = x.reshape(x.shape[0],self.cha_1,
#                           self.cha_1_reshape)

#             x = self.batch_norm_c1(x)
#             x = self.dropout_c1(x)
#             x = F.relu(self.conv1(x))

#             x = self.ave_po_c1(x)

#             x = self.batch_norm_c2(x)
#             x = self.dropout_c2(x)
#             x = F.relu(self.conv2(x))
#             x_s = x

#             x = self.batch_norm_c2_1(x)
#             x = self.dropout_c2_1(x)
#             x = F.relu(self.conv2_1(x))

#             x = self.batch_norm_c2_2(x)
#             x = self.dropout_c2_2(x)
#             x = F.relu(self.conv2_2(x))
#             x =  x * x_s

#             x = self.max_po_c2(x)

#             x = self.flt(x)

#             x = self.batch_norm3(x)
#             x = self.dropout3(x)
#             x = self.dense3(x)

#             return x

In [13]:
class ConvFeatureExtractions(nn.Module):
    def __init__(self, num_features, hidden_size, channel_1=256, channel_2=512):
        super().__init__()
        
        self.channel_1 = channel_1
        self.channel_2 = channel_2
        self.final_conv_features = int(hidden_size / channel_1) * channel_2
            
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.15)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
               
        self.batch_norm_c1 = nn.BatchNorm1d(channel_1)
        self.dropout_c1 = nn.Dropout(0.15)
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(channel_1,channel_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

        self.batch_norm_c2 = nn.BatchNorm1d(channel_2)
        self.dropout_c2 = nn.Dropout(0.2)
        self.conv2 = nn.utils.weight_norm(nn.Conv1d(channel_2,channel_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)
        
        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)
        self.final_conv_features = int(self.final_conv_features / 2)
        
        self.flt = nn.Flatten()
     
    def forward(self, x):
        
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        x = x.reshape(x.shape[0], self.channel_1, -1)
        
        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        
        x = self.max_po_c2(x)
        
        x = self.flt(x)
        
        return x
        


class MoaModel_V4(nn.Module):
        def __init__(self, model_config):
            super(MoaModel_V4, self).__init__()
            hidden_size = 512
            dropout = 0.15
            
            self.gene_cnn_features = ConvFeatureExtractions(model_config.number_of_genes, 2048, channel_1=128, channel_2=256)
            self.cell_cnn_features = ConvFeatureExtractions(model_config.number_of_cells, 1024, channel_1=64, channel_2=128)
            
            encoded_features = self.gene_cnn_features.final_conv_features + self.cell_cnn_features.final_conv_features
            
            self.block1 = MoaModelBlock(encoded_features, hidden_size, dropout, weight_norm=True)
            self.model = MoaEncodeBlock(hidden_size, model_config.number_of_targets, dropout, weight_norm=True)
            

        def forward(self, data):
            
            x_genes = data['genes'].to(DEVICE)
            x_cells = data['cells'].to(DEVICE)

            x_genes = self.gene_cnn_features(x_genes)
            x_cells = self.cell_cnn_features(x_cells)
            
            x = torch.cat((x_genes, x_cells), dim=1)

            x = self.block1(x)
            x = self.model(x)

            return x

# Smooth loss function

In [14]:
class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets, n_classes, smoothing=0.0):
        assert 0 <= smoothing <= 1
        with torch.no_grad():
            targets = targets * (1 - smoothing) + torch.ones_like(targets).to(DEVICE) * smoothing / n_classes
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss()._smooth(targets, inputs.shape[1], self.smoothing)

        if self.weight is not None:
            inputs = inputs * self.weight.unsqueeze(0)

        loss = F.binary_cross_entropy_with_logits(inputs, targets)

        return loss
    
    
class SmoothBCEwLogits(_WeightedLoss):
        def __init__(self, weight=None, reduction='mean', smoothing=0.0,pos_weight = None):
            super().__init__(weight=weight, reduction=reduction)
            self.smoothing = smoothing
            self.weight = weight
            self.reduction = reduction
            self.pos_weight = pos_weight

        @staticmethod
        def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
            assert 0 <= smoothing < 1
            with torch.no_grad():
                targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            return targets

        def forward(self, inputs, targets):
            targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
                self.smoothing)
            loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight,
                                                      pos_weight = self.pos_weight)

            if  self.reduction == 'sum':
                loss = loss.sum()
            elif  self.reduction == 'mean':
                loss = loss.mean()

            return loss

# Scaling functions

In [15]:
def true_rank_gaus_scale(data, columns): 
    global DEVICE
    
    if DEVICE == 'cuda':
        import cupy as cp
        from cupyx.scipy.special import erfinv
        epsilon = 1e-6

        for f in columns:
            r_gpu = cp.array(data[f].values)
            r_gpu = r_gpu.argsort().argsort()
            r_gpu = (r_gpu/r_gpu.max()-0.5)*2 
            r_gpu = cp.clip(r_gpu,-1+epsilon,1-epsilon)
            r_gpu = erfinv(r_gpu) 
            data[f] = cp.asnumpy( r_gpu * np.sqrt(2) )
            
        return data
    
    from scipy.special import erfinv as sp_erfinv
    
    epsilon = 1e-6
    
    for f in columns:
        r_cpu = data[f].values.argsort().argsort()
        r_cpu = (r_cpu/r_cpu.max()-0.5)*2 
        r_cpu = np.clip(r_cpu,-1+epsilon,1-epsilon)
        r_cpu = sp_erfinv(r_cpu) 
        data[f] = r_cpu * np.sqrt(2)
        
    return data

#THIS IS NOT CORRECT SINCE IT NEEDS TO BE SCALED ONLY BY TRAIN DATA
# def quantile_scale_dosetime(data, columns):
#     global RANDOM_SEED
#     arr = []
    
#     for cp_dose in ['D1', 'D2']:
#         for cp_time in [24, 48, 72]:
#             temp_data = data[data.cp_dose == cp_dose].reset_index(drop=True)
#             temp_data = temp_data[temp_data.cp_time == cp_time].reset_index(drop=True)

#             scaler = QuantileTransformer(n_quantiles=100,random_state=RANDOM_SEED, output_distribution="normal")
#             temp_data[columns] = scaler.fit_transform(temp_data[columns])
#             arr.append(temp_data)
            
#     return pd.concat(arr).reset_index(drop=True)

def quantile_dosetime_scaling(train_data, valid_data, test_data, feature_columns):
    global RANDOM_SEED
    
    train_arr = []
    valid_arr = []
    test_arr = []
    
    for cp_dose in ['D1', 'D2']:
        for cp_time in [24, 48, 72]:
            temp_train = train_data[train_data.cp_dose == cp_dose].reset_index(drop=True)
            temp_train = temp_train[temp_train.cp_time == cp_time].reset_index(drop=True)
            
            temp_valid = valid_data[valid_data.cp_dose == cp_dose].reset_index(drop=True)
            temp_valid = temp_valid[temp_valid.cp_time == cp_time].reset_index(drop=True)
            
            temp_test = test_data[test_data.cp_dose == cp_dose].reset_index(drop=True)
            temp_test = temp_test[temp_test.cp_time == cp_time].reset_index(drop=True)

            scaler = QuantileTransformer(n_quantiles=100,random_state=RANDOM_SEED, output_distribution="normal")
            temp_train[feature_columns] = scaler.fit_transform(temp_train[feature_columns])
            temp_valid[feature_columns] = scaler.transform(temp_valid[feature_columns])
            temp_test[feature_columns] = scaler.transform(temp_test[feature_columns])
            
            train_arr.append(temp_train)
            valid_arr.append(temp_valid)
            test_arr.append(temp_test)
            
    train_data = pd.concat(train_arr).reset_index(drop=True)
    valid_data = pd.concat(valid_arr).reset_index(drop=True)
    test_data = pd.concat(test_arr).reset_index(drop=True)
            
    return train_data, valid_data, test_data

def true_rankgaus_dosetime(data, columns):
    global RANDOM_SEED
    arr = []
    
    for cp_dose in ['D1', 'D2']:
        for cp_time in [24, 48, 72]:
            temp_data = data[data.cp_dose == cp_dose].reset_index(drop=True)
            temp_data = temp_data[temp_data.cp_time == cp_time].reset_index(drop=True)
            
            arr.append(true_rank_gaus_scale(temp_data, columns))
        
    return pd.concat(arr).reset_index(drop=True)

def true_rankgaus_dosetime_scaling(train_data, valid_data, test_data, feature_columns):
    
    train_data = true_rankgaus_dosetime(train_data, feature_columns)
    valid_data = true_rankgaus_dosetime(valid_data, feature_columns)
    test_data = true_rankgaus_dosetime(test_data, feature_columns)
    
    return train_data, valid_data, test_data
    

def true_rankgaus_scaling(train_data, valid_data, test_data, feature_columns):
    
    train_data = true_rank_gaus_scale(train_data, feature_columns)
    valid_data = true_rank_gaus_scale(valid_data, feature_columns)
    test_data = true_rank_gaus_scale(test_data, feature_columns)
    
    return train_data, valid_data, test_data
    

def quantile_scaling(train_data, valid_data, test_data, feature_columns):
    global RANDOM_SEED
    
    scaler = QuantileTransformer(n_quantiles=100,random_state=RANDOM_SEED, output_distribution="normal")
    train_data[feature_columns] = scaler.fit_transform(train_data[feature_columns])
    valid_data[feature_columns] = scaler.transform(valid_data[feature_columns])
    test_data[feature_columns] = scaler.transform(test_data[feature_columns])
    
    return train_data, valid_data, test_data

#THIS SHOULD NOT BE CORRECT
# def quantile_dosetime_scaling(train_data, valid_data, test_data):
    
#     global feature_columns
    
#     train_data = quantile_scale_dosetime(train_data, feature_columns)
#     valid_data = quantile_scale_dosetime(valid_data, feature_columns)
#     test_data = quantile_scale_dosetime(test_data, feature_columns)
    
#     return train_data, valid_data, test_data

# Data preprocessing

In [16]:
def create_dataloader(data, batch_size, shuffle, target_columns=None):
    gene_features = [c for c in data.columns if 'g-' in c]
    cell_features = [c for c in data.columns if 'c-' in c]
    
    dataset = MoaDataset(data, gene_features, cell_features, target_columns)
    
    return torch.utils.data.DataLoader(dataset,
                                       batch_size=batch_size,
                                       shuffle=shuffle)

In [17]:
def pca_transform(fitted_pca, data, feature_columns, sig_ids, base_feature_name):
    feature_data = fitted_pca.transform(data[feature_columns].values)
    df = pd.DataFrame(feature_data, columns =[f'{base_feature_name}-{i}' for i in range(feature_data.shape[1])])
    df['sig_id'] = sig_ids
    return df


def preprocess_fold_data(train_data, test_data, fold, scaling_func=None, add_pca=False):
    global feature_columns, target_columns, gene_features, cell_features
    
    fold_train_data = train_data[train_data.fold != fold].reset_index(drop=True)
    fold_valid_data = train_data[train_data.fold == fold].reset_index(drop=True)
    
    if add_pca:
        fold_data = [fold_train_data, fold_valid_data, test_data]
        
        pca_genes = PCA(n_components=150)
        pca_cells = PCA(n_components=30)
        
        pca_genes.fit(fold_train_data[gene_features].values)
        pca_cells.fit(fold_train_data[cell_features].values)
        
        for fitted_pca, pca_features, colum_name in [(pca_genes, gene_features, 'g-pca'), (pca_cells, cell_features, 'c-pca')]:
            for i, pca_data in enumerate(fold_data):
                fitted_pca_data = pca_transform(fitted_pca=fitted_pca, 
                                                data=pca_data, 
                                                feature_columns=pca_features, 
                                                sig_ids=pca_data.sig_id.values, 
                                                base_feature_name=colum_name)
                fold_data[i] = pd.merge(fold_data[i], fitted_pca_data, on='sig_id')
            
           
        fold_train_data = fold_data[0]
        fold_valid_data = fold_data[1]
        test_data = fold_data[2]
    
    if scaling_func is not None:
        fold_train_data, fold_valid_data, test_data = scaling_func(fold_train_data, fold_valid_data, test_data, feature_columns)
      
    train_dataloader = create_dataloader(data=fold_train_data, batch_size=BATCH_SIZE, shuffle=True, target_columns=target_columns)
    valid_dataloader = create_dataloader(data=fold_valid_data, batch_size=BATCH_SIZE, shuffle=False, target_columns=target_columns)
    test_dataloader = create_dataloader(data=test_data, batch_size=BATCH_SIZE, shuffle=False, target_columns=None)
    
    return train_dataloader, valid_dataloader, test_dataloader     

# Blending functions

In [18]:
def log_loss_numpy(y_pred):
    loss = 0
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    for i in range(y_pred.shape[1]):
        loss += - np.mean(y_true[:, i] * np.log(y_pred_clip[:, i]) + (1 - y_true[:, i]) * np.log(1 - y_pred_clip[:, i]))
    return loss / y_pred.shape[1]

def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof, axes = ((0), (0)))
    score = log_loss_numpy(oof_blend)
    
    coef = 1e-6
    penalty = coef * (np.sum(weights) - 1) ** 2
    return score + penalty

def grad_func(weights):
    oof_clip = np.clip(oof, 1e-15, 1 - 1e-15)
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof_clip[i], 0
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
    return gradients

oof = []
y_true = []
def find_optimal_blend(predictions, train_data, target_columns):
    
    global oof, y_true
    y_true = train_data.sort_values(by='sig_id')[target_columns].values
    oof = np.zeros((len(predictions), y_true.shape[0], y_true.shape[1]))

    for i, pred in enumerate(predictions):
        oof[i] = pred.sort_values(by='sig_id')[target_columns].values

    tol = 1e-10
    init_guess = [1 / oof.shape[0]] * oof.shape[0]
    bnds = [(0, 1) for _ in range(oof.shape[0])]
    cons = {'type': 'eq', 
            'fun': lambda x: np.sum(x) - 1, 
            'jac': lambda x: [1] * len(x)}

    res_scipy = minimize(fun = func_numpy_metric, 
                         x0 = init_guess, 
                         method = 'SLSQP', 
                         jac = grad_func, 
                         bounds = bnds, 
                         constraints = cons, 
                         tol = tol)
    
    return res_scipy.x

# Utils functions

In [19]:
def inference(model, data_loader, target_columns):
    predictions = []
    
    model.eval()

    for batch in data_loader:
        batch_predictions = model(batch).sigmoid().detach().cpu().numpy()
        sig_ids = np.array(batch['sig_id'])

        df = pd.DataFrame(batch_predictions, columns=target_columns)
        df['sig_id'] = sig_ids
        predictions.append(df)

    return pd.concat(predictions).reset_index(drop=True)

In [20]:
def calculate_log_loss(predicted_df, train_df, target_columns):
    predicted_df = predicted_df.copy()
    train_df = train_df.copy()
    
    predicted_df = predicted_df[target_columns + ['sig_id']].reset_index(drop=True)
    predicted_df = predicted_df.sort_values(by=['sig_id'])
    predicted_df = predicted_df.drop('sig_id', axis=1)

    true_df = train_df[target_columns + ['sig_id']].reset_index(drop=True)
    true_df = true_df.sort_values(by=['sig_id'])
    true_df = true_df.drop('sig_id', axis=1)

    predicted_values = predicted_df.values
    true_values = true_df.values
    
    score = 0
    loss_per_class = []
    for i in range(predicted_values.shape[1]):
        _score = log_loss(true_values[:, i].astype(np.float), predicted_values[:, i].astype(np.float), eps=1e-15, labels=[1,0])
        loss_per_class.append(_score)
        score += _score / predicted_values.shape[1]

    return score, loss_per_class

def scale_predictions(predictions, target_columns, scale_values=None):
    predictions = [p.copy() for p in predictions]
    predictions = [p.sort_values(by=['sig_id']).reset_index(drop=True) for p in predictions]
    
    final_predictions = np.zeros((predictions[0].shape[0], len(target_columns)))
    
    for i, p in enumerate(predictions):
        p_values = p[target_columns].values
        
        if scale_values is None:
            final_predictions += p_values / len(predictions)
        else:
            final_predictions += (p_values * scale_values[i])
        
    predictions_df = predictions[0].copy()
    predictions_df.loc[:, target_columns] = final_predictions
    
    return predictions_df

In [21]:
class TrainFactory:
    
    @classmethod
    def model_version1(cls, train_loader, epochs):
        global model_config, DEVICE
        
        model = MoaModel_V1(model_config).to(DEVICE)
        best_model = MoaModel_V1(model_config).to(DEVICE)
        
#         optimizer = torch.optim.Adam(params=model.parameters(),
#                                      lr=1e-3,
#                                      weight_decay=3e-5)
        
#         scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
#                                                         max_lr=5e-2,
#                                                         epochs=epochs, 
#                                                         steps_per_epoch=len(train_loader))
        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.04647353847564317, weight_decay=8.087569236449597e-06)

        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_loader)*epochs//2, num_training_steps=len(train_loader)*epochs)
        
        loss_fn = nn.BCEWithLogitsLoss()
        
        return model, best_model, optimizer, scheduler, loss_fn
    
    
    @classmethod
    def model_version2(cls, train_loader, epochs):
        global model_config, DEVICE
        
        model = MoaModel_V2(model_config).to(DEVICE)
        best_model = MoaModel_V2(model_config).to(DEVICE)
        
        optimizer = torch.optim.Adam(params=model.parameters(),
                                     lr=1e-3,
                                     weight_decay=1e-5)
        
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                        max_lr=1e-2,
                                                        epochs=epochs, 
                                                        steps_per_epoch=len(train_loader))
        
        loss_fn = nn.BCEWithLogitsLoss()
        
        return model, best_model, optimizer, scheduler, loss_fn
    
    
    @classmethod
    def model_version3(cls, train_loader, epochs):
        global model_config, DEVICE
        
        model = MoaModel_V3(model_config).to(DEVICE)
        best_model = MoaModel_V3(model_config).to(DEVICE)
        
        optimizer = torch.optim.Adam(params=model.parameters(),
                                     lr=1e-3,
                                     weight_decay=1e-5)
        
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                        max_lr=1e-2,
                                                        epochs=epochs, 
                                                        steps_per_epoch=len(train_loader))

        loss_fn = nn.BCEWithLogitsLoss()
        
        return model, best_model, optimizer, scheduler, loss_fn
    
    
    @classmethod
    def model_version4(cls, train_loader, epochs):
        global model_config, DEVICE
        
        model = MoaModel_V4(model_config).to(DEVICE)
        best_model = MoaModel_V4(model_config).to(DEVICE)
        
        optimizer = torch.optim.Adam(params=model.parameters(),
                                     lr=1e-3,
                                     weight_decay=1e-5)
        
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                        max_lr=5e-3,
                                                        epochs=epochs, 
                                                        steps_per_epoch=len(train_loader))

#         optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
#         scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                                   max_lr=1e-2, epochs=epochs, steps_per_epoch=len(train_loader))
        
        loss_fn = nn.BCEWithLogitsLoss()

        return model, best_model, optimizer, scheduler, loss_fn

In [22]:
def train_model(model, best_model, optimizer, scheduler, loss_fn, train_loader, valid_loader, test_loader, epochs):
    global gene_features, cell_features, target_columns
    
    train_data = train_loader.dataset.dataset_df
    valid_data = valid_loader.dataset.dataset_df
    
    best_loss = np.inf
    
    for epoch in range(epochs):
        
        model.train()
        train_loss = 0
        
        for train_batch in train_loader:
            optimizer.zero_grad()
            
            y_pred = model(train_batch)
            y_true = train_batch['y'].to(DEVICE)
            
            curr_train_loss = loss_fn(y_pred, y_true)
            
            curr_train_loss.backward()
            
            optimizer.step()
            scheduler.step()
            
            train_loss += ( curr_train_loss.item() * (len(train_batch['sig_id']) / len(train_data)))
            
            
        valid_predictions = inference(model, valid_loader, target_columns)
        valid_loss, _ = calculate_log_loss(valid_predictions, valid_data, target_columns)
        
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_model.load_state_dict(model.state_dict())
            
                           
        if (epoch + 1) % 5 == 0:
            print(f'Epoch:{epoch} \t train_loss:{train_loss:.10f} \t valid_loss:{valid_loss:.10f}')
            
    
    valid_predictions = inference(best_model, valid_loader, target_columns)
    test_predictions = inference(best_model, test_loader, target_columns)
    
    return best_model, valid_predictions, test_predictions

In [23]:
#Hyperparameters
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

FOLDS = 5
EPOCHS = 30
BATCH_SIZE = 128
SEEDS = [11, 221, 50]

In [24]:
#Creating the cross validation strategy
train_data = create_cross_validation_strategy(train_data, target_columns, FOLDS, RANDOM_SEED)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [25]:
train_data = train_data[train_data.cp_type == 'trt_cp'].reset_index(drop=True)

In [26]:
class ModelTrainConfig:
    def __init__(self, model_name, factory_func, scaling_func, add_pca):
        self.model_name = model_name
        self.factory_func = factory_func
        self.scaling_func = scaling_func
        self.add_pca = add_pca

In [27]:
model_version1 = ModelTrainConfig(model_name='version_1', 
                                  factory_func=TrainFactory.model_version1, 
                                  scaling_func=true_rankgaus_scaling,
                                  add_pca=False)

model_version2 = ModelTrainConfig(model_name='version_2', 
                                  factory_func=TrainFactory.model_version2, 
                                  scaling_func=quantile_dosetime_scaling,
                                  add_pca=False)

model_version3 = ModelTrainConfig(model_name='version_3', 
                                  factory_func=TrainFactory.model_version3, 
                                  scaling_func=quantile_scaling,
                                  add_pca=False)

model_version4 = ModelTrainConfig(model_name='version_4', 
                                  factory_func=TrainFactory.model_version4, 
                                  scaling_func=quantile_scaling,
                                  add_pca=False)

models_train_configs = [model_version1, model_version2, model_version3, model_version4]
#models_train_configs = [model_version4]

In [28]:
models_valid_predictions = []
models_test_predictions = []

seed_losses = []

for model_train_config in models_train_configs:
    print(f'Training model:{model_train_config.model_name}')
    
    single_model_valid_predictions = []
    single_model_test_predictions = []
    
    for seed in SEEDS:
        seed_everything(seed)

        model_seed_valid_predictions = []
        model_seed_test_predictions = []

        for fold in range(FOLDS):
            print(f'Training fold: {fold}')

            fold_test_data = test_data[test_data.cp_type == 'trt_cp'].reset_index(drop=True)

            train_loader, valid_loader, test_loader = preprocess_fold_data(train_data=train_data, 
                                                                           test_data=fold_test_data, 
                                                                           fold=fold, 
                                                                           scaling_func=model_train_config.scaling_func)
            
            number_of_genes, number_of_cells = train_loader.dataset.number_of_features()

            model_config = ModelConfig(number_of_genes=number_of_genes, 
                                       number_of_cells=number_of_cells, 
                                       number_of_targets=len(target_columns))

            model, best_model, optimizer, scheduler, loss_fn = model_train_config.factory_func(train_loader, EPOCHS)

            best_model, valid_predictions, test_predictions = train_model(model=model,
                                                                          best_model=best_model,
                                                                          optimizer=optimizer, 
                                                                          scheduler=scheduler, 
                                                                          loss_fn=loss_fn, 
                                                                          train_loader=train_loader, 
                                                                          valid_loader=valid_loader, 
                                                                          test_loader=test_loader, 
                                                                          epochs=EPOCHS)

            #TODO: Save the model here.
            torch.save(best_model.state_dict(), f'model-{model_train_config.model_name}_fold-{fold}_seed-{seed}')
            
            model_seed_valid_predictions.append(valid_predictions)
            model_seed_test_predictions.append(test_predictions)
            print('-' * 100)


        valid_predictions = pd.concat(model_seed_valid_predictions).reset_index(drop=True)
        test_predictions = scale_predictions(model_seed_test_predictions, target_columns)
        
        single_model_valid_predictions.append(valid_predictions)
        single_model_test_predictions.append(test_predictions)
        
        valid_loss, _ = calculate_log_loss(valid_predictions, train_data, target_columns)

        seed_losses.append(valid_loss)

        print(f'Model:{model_train_config.model_name} \t Seed:{seed} \t oof_loss:{valid_loss:.10f}')

    valid_predictions = scale_predictions(single_model_valid_predictions, target_columns)
    test_predictions = scale_predictions(single_model_test_predictions, target_columns)
    
    models_valid_predictions.append(valid_predictions)
    models_test_predictions.append(test_predictions)
    
    valid_loss, _ = calculate_log_loss(valid_predictions, train_data, target_columns)

    print(f'Model:{model_train_config.model_name} \t valid_loss:{valid_loss:.10f}')

          

Training model:version_1
Training fold: 0
Epoch:4 	 train_loss:0.0186646465 	 valid_loss:0.0187850279
Epoch:9 	 train_loss:0.0189539609 	 valid_loss:0.0195378892
Epoch:14 	 train_loss:0.0204105963 	 valid_loss:0.0203869686
Epoch:19 	 train_loss:0.0201283952 	 valid_loss:0.0195157384
Epoch:24 	 train_loss:0.0186819164 	 valid_loss:0.0183624131
Epoch:29 	 train_loss:0.0161599823 	 valid_loss:0.0170575871
----------------------------------------------------------------------------------------------------
Training fold: 1
Epoch:4 	 train_loss:0.0247784558 	 valid_loss:0.0207367345
Epoch:9 	 train_loss:0.0187313250 	 valid_loss:0.0193012083
Epoch:14 	 train_loss:0.0199654153 	 valid_loss:0.0205130168
Epoch:19 	 train_loss:0.0198497827 	 valid_loss:0.0205473328
Epoch:24 	 train_loss:0.0183039755 	 valid_loss:0.0186049616
Epoch:29 	 train_loss:0.0156739520 	 valid_loss:0.0174691290
----------------------------------------------------------------------------------------------------
Training fo

In [29]:
#Finding optimal blend weights
blend_weights = find_optimal_blend(models_valid_predictions, train_data, target_columns)

print(f'Optimal blend weights: {blend_weights}')

Optimal blend weights: [0.24941872 0.21155631 0.14350498 0.39551999]


In [30]:
valid_predictions = scale_predictions(models_valid_predictions, target_columns, blend_weights)
test_predictions = scale_predictions(models_test_predictions, target_columns, blend_weights)

In [31]:
validation_loss, _ = calculate_log_loss(valid_predictions, train_data, target_columns)
print(f'Validation loss: {validation_loss}')

Validation loss: 0.016912176731223444


In [32]:
print(f'Seed loss std: {np.array(seed_losses).std():.10f}')

Seed loss std: 0.0000571888


In [33]:
zero_ids = test_data[test_data.cp_type == 'ctl_vehicle'].sig_id.values

zero_df = pd.DataFrame(np.zeros((len(zero_ids), len(target_columns))), columns=target_columns)
zero_df['sig_id'] = zero_ids

nonzero_df = test_predictions[~test_predictions.sig_id.isin(zero_ids)]
nonzero_df = nonzero_df[target_columns + ['sig_id']].reset_index(drop=True)
submission = pd.concat([nonzero_df, zero_df])

In [34]:
submission.to_csv('submission.csv', index=False)

In [35]:
submission.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,sig_id
0,0.000814,0.000792,0.002127,0.014007,0.020311,0.004433,0.002258,0.004669,0.000236,0.008374,...,0.001330,0.003498,0.001483,0.000942,0.000530,0.000940,0.001786,0.002402,0.001530,id_0004d9e33
1,0.000448,0.000667,0.001105,0.002065,0.001819,0.001578,0.004356,0.009554,0.005368,0.015163,...,0.001039,0.002600,0.000562,0.011257,0.000347,0.014713,0.000895,0.002473,0.003129,id_001897cda
2,0.000617,0.000714,0.001744,0.015345,0.018921,0.004220,0.002807,0.005043,0.000266,0.007766,...,0.001259,0.002291,0.008163,0.005422,0.000419,0.001800,0.001906,0.000966,0.001985,id_00276f245
3,0.001374,0.001135,0.001435,0.016436,0.024293,0.004680,0.003772,0.002728,0.000357,0.013833,...,0.000648,0.003238,0.001507,0.001197,0.000522,0.001046,0.001740,0.000500,0.001596,id_0027f1083
4,0.000691,0.000689,0.001790,0.022545,0.025744,0.004296,0.003590,0.002675,0.000276,0.010715,...,0.001037,0.002978,0.004684,0.001436,0.000443,0.000775,0.002105,0.000404,0.001410,id_006fc47b8
